<a href="https://colab.research.google.com/github/shineloveyc/Doing_ML/blob/master/Topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Latent Dirichlet Allocation(LDA) for topic modeling

In [0]:
#load the library
import pandas as pd
import tarfile
from pathlib import Path
#import pyprind #python progress indicator
import os
from fastai.vision import *

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#set up the path to read data
path = Path(base_dir + 'data/')
print(path)

/content/gdrive/My Drive/Colab Notebooks/data


In [0]:
#check current working directory
os.chdir(path)

In [0]:
os.getcwd()

'/content/gdrive/My Drive/Colab Notebooks/data'

In [0]:
df = pd.read_csv('movie_data.csv', encoding = 'utf-8')
df.head(3)

,review,sentiment
0,I just finished watching this movie and I must...,1
1,Dustin Hoffman's debut feature isn't as bad as...,0
2,"One of the best ""Amitabh comeback"" movies I li...",1


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df = 0.1,
                        max_features = 5000)
#max_features: top 5000 words, max_df: freq larger than threashold will be exclude

X = count.fit_transform(df['review'].values)

In [0]:
X.shape

(50000, 5000)

In [0]:
#learn the topic
from sklearn.decomposition import LatentDirichletAllocation

lad = LatentDirichletAllocation(n_components = 10,
                                random_state = 123,
                                learning_method = 'batch') #setting to bacth means train all data at once, alternativ is 'online'

X_topics = lad.fit_transform(X)

In [0]:
X_topics.shape

(50000, 10)

In [0]:
#store a matrix containing the word importance(5000) for each of the 10 topics in increasing order
lad.components_.shape

(10, 5000)

In [0]:
#print five most important words for each of the 10 topics
n_top_words =5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lad.components_):
  print("Topic %d:" % (topic_idx + 1))
  print(" ".join([feature_names[i] for i in topic.argsort()\  #sort the word in each topic in a reverse order, becasue the the importance value sorted in increasing order
                  [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
war american dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl blood
Topic 7:
role performance comedy actor performances
Topic 8:
series episode episodes tv season
Topic 9:
book version original effects fi
Topic 10:
action guy fight guys cool


In [0]:
#horror topic is in index 5
horror = X_topics[:, 5].argsort()[::1]

#print first 300 characters from the top three horror movies
for iter_idx, movie_idx in enumerate(horror[:3]):
  print('\nHorror movie #%d:' % (iter_idx + 1))
  print(df['review'][movie_idx][:300], '...')


Horror movie #1:
Match 1: Tag Team Table Match Bubba Ray and Spike Dudley vs Eddie Guerrero and Chris Benoit Bubba Ray and Spike Dudley started things off with a Tag Team Table Match against Eddie Guerrero and Chris Benoit. According to the rules of the match, both opponents have to go through tables in order to get ...

Horror movie #2:
(Some spoilers included:)<br /><br />Although, many commentators have called this film surreal, the term fits poorly here. To quote from Encyclopedia Britannica's, surreal means:<br /><br />"Fantastic or incongruous imagery": One needn't explain to the unimaginative how many ways a plucky ten-year-o ...

Horror movie #3:
Titanic directed by James Cameron presents a fictional love story on the historical setting of the Titanic. The plot is simple, noncomplicated, or not for those who love plots that twist and turn and keep you in suspense. The end of the movie can be figured out within minutes of the start of the fil ...
